**Disclaimer**: los pasos indicados para alcanzar un modelo de datos no tienen por qué realizarse necesariamente en el orden propuesto. Muchas etapas podrían cambiarse de orden o superponerse. Esta guía sólo propone un camino para discutir diferentes aspectos que nos puede permitir alcanzar un resultado final adaptado a nuestro contexto.

# Modelando un dominio con RDF

## Paso 1: Preguntas sobre el alcance y la estructura del conocimiento.

*   ¿Cuáles serán las clases de mi dominio?
*   ¿Es posible que mi contexto inicial se expanda enun futuro? ¿Cómo afecta eso a mi modelo?
*   ¿Qué características voy a modelar sobre cada una de las entidades de mi dominio?
* ¿Cómo deberían relacionarse las entidades de mi dominio?
* Lo que he detectado como instancia en mi domino, es en realidad (o puede actuar como) una clase?

Estas preguntas no son, en esencia, muy distintas de las preguntas que nos hacemos a la hora de comenzar a modelar una base de datos relacional. Pero debemos adaptar nuestras respuestas y manera de pensar al paradigma RDF.
En esta primera etapa, tendríamos que obtener un producto que nos permita comprender las topología (las estructuras) que se darán en nuestros datos. Aunque, probablemente, será pronto para hacerse preguntas de caracter más técnico, tales como ontologías/tecnologías a usar para almacenar y exponer la información objetivo.



## Dominio de ejemplo: Pokemon

**Clases candidatas:**


*   Especie de pokemon
*   Tipo de pokemon
*   Objeto
*   Máquina Técnica
*   Máquina Oculta
*   Pokeball
*   Movimiento
*   Medalla
*   Gimnasio
* ...

### **Posibles vías de expansión:**
Las clases mencionadas anteriormente parten de una noción abstracta sobre conceptos incluidos en el lore del universo pokemon. Pero ese universo se puede manifestar en distintos contextos, afectando al modelo y alcance de mis datos:

* ¿Películas y series? Aparecería nociones como temporada, episodio, película, personajes concretos que aparecen en uno u otro lugar...
*  ¿Videojuegos? Títulos, plataformas, distintas versiones de Pokedex, información contradictoria para las especies...
*  ¿Juegos de cartas? Necesidad de rediseñar la noción de Movimiento: su coste, su efectividad, como se asocia a cada especie...


### **Características sobre cada entidad:**

Aunque en esta primara fase solemos trabajar a un nivel más bien abstracto, en este caso ayuda centrarse en la observación de individuos (instancias) concretas. Por ejemplo, cojamos a Charmander. ¿Qué podemos decir sobre él?
* Su tipo: fuego.
* Su descripción: entrada de la Pokedex.
* Tiene ciertas estadísticas base (varias).
* Tiene habilidades (base y oculta).
* Tiene varias características físicas asociadas (color, altura, peso, etc.)

* Evoluciona en otro Pokemon: Charmeleon (¿de qué forma?).
* Puede aprender ciertos movimientos (¿en qué nivel o condiciones?).
* ...

Cogemos otra tipo de entidad. Por ejemplo, un movimiento: Rayo. ¿Qué podemos decir sobre él?
* Tiene un tipo: eléctrico.
* Tiene una descripción: texto.
* Tiene una potencia.
* Tiene una precisión.
* ¿Tiene prioridad?
* ¿Puede causar un cambio de estado? ¿Cuál? ¿Con qué probabilidad?
* ¿Tiene alguna otra característica especial?
* Tiene unos pokemon que lo pueden aprender (¿en qué condiciones?).


Aparecen muchas preguntas siempre que modelamos cualquier entidad.


* Cuando una noción se vuelve compleja y escapa la sencillez de sujeto-predicado-objeto para ser representada de forma completa (ejemplo: Charmander evoluciona a Charmeleon, pero ¿cómo?) tenemos que tomar la decisión de adoptar estructuras más complejas o renunciar a cierto grado de expresividad.

* Cuando dos tipos de entidades tienen una relación (ejemplo: cierto poekmon aprende cierto movimiento) hemos de plantearnos tratar de no repetir información y plantearnos la dirección más natural de esta relación (el pokemon APRENDE el movimiento vs el movimiento ES APRENDIDO por el pokemon).

* Cuando hay clases disfrazadas de instancia (ejemplo: charmander es clase o instancia). Debemos ser lo más pulcros posible, pero sin olvidar que, en realidad, no hay une necesidad estricta de que cierta URI se comporte exclusivamente como clase o como instancia. Si no va a haber razonamiento lógico con los datos, la decisión debería ser guiada, sobre todo, tratando de hacer el modelo lo más natural que sea posible. Si va a haber razonamiento, deberíamos tener en cuanta nociones referidas a OWL punning y sus implicaciones.

* Tenemos que ser conscientes de cuál es el objetivo de nuestro grafo. ¿Queremos dar soporte a cierto caso de uso? ¿O queremos modelar con brillantex todo lo que se pueda modelar en mi contexto? A partir de cierto punto, modelar cierta parte del dominio puede volverse muy complejo. Ese esfuerzo debería llevarse a cabo sólo si es necesario.



In [4]:
#Ejemplos de distintas representaciones, más o menos expresivas, complejas o correctas, para indicar que Charmander evoluciona
# en Charmeleon.

# Correcto, pero perdemos mucha información
tt1_1 = """
:charmander :evoluciona_a :charmeleon .
"""

# Tenemos más información, pero perder una porpiedad general como :evoluciona hará que mis consultas SPARQL sean más complejas.
# Necesito saber el tipo de evolución de un pokemon para saber a quien evoluciona mediante una consulta
ttl_2 =  """
:charmander :evoluciona_por_nivel  :charmeleon .
"""

# Más información que antes, pero problemas MUCHO mayores de representación. No es ranonable crear una propiedad por cada
# nivel en que alguien pueda evolucionar.
ttl_3 = """
:charmander :evoluciona_nivel_16  :charmeleon .
"""

# Mucho más complejo, no existe vinculo directo entre charmander y charmeleon, mis consultas se me complican. Pero no he
# perdido ninguna información y las propiedades tienen sentido. Aparece un nodo llamado evolucion_charmander que no
# forma parte de mi dominio real, es un elemento artificial que me permite hablar unir nociones complejas.
ttl_4 = """
:charmander :tiene  :evolucion_charmander

:evolucion_charmander  a :evolucion ;
                       :evoluciona_ a :charmeleon ;
                       :tipo_evolucon  :por_nivel ;
                       :nivel_evolucion  16  ;
"""

# Esquema parecido al anterior, pero no hacemos que charmander apunte a un nodo inexistente en la realidad. La
# información de esa evolución la colocamos toda en un nodo anónimo que apunta tanto a charmander como a charmeleon.
# Tiene sentido que este nodo sea anónimo, porque no es una entidad que exista. Pero ojo, si hacemos esto, tenemos que
# tener en cuenta que los nodos anónimos sólo tienen sentido en el contexto del propio grafo. Podemos referenciar al nodo
#  _:charmander_charmeleon desde nuestro datos, pero No podremos hacer referencia al nodo _:charmander_charmeleon desde
# otra fuente de datos.
ttl_5 = """

_:charmander_charmeleon    a :evolucion ;
                           :desde  :charmander ;
                           :hacia  :charmeleon ;
                           :tipo_evolucon  :por_nivel ;
                           :nivel_evolucion  16  ;
"""


# Exactamente la misma información con la misma estructura que en el caso anterior, pero usando una sintaxis alternativa.
# Así la escritura es más compacta, pero ojo, perdemos la capacidad de referenciar desde nuestro propio grafo al nodo que
# antes se llamaba :charmander_charmeleon. Con esta sintaxis no tiene ningún identificador, es aún "más anónimo"
ttl_6 = """

[  a :evolucion ;
   :desde  :charmander ;
   :hacia  :charmeleon ;
   :tipo_evolucon  :por_nivel ;
   :nivel_evolucion  16  ;
]

"""



## Paso 2: Conexión con otras ontologías o grafos de conocimiento

Una vez tenemos claro el alcance y la estructura de lo que queremos representar, es hora de hacerse otras preguntas que tienen que ver con prevenir esfuerzos innecesarios e incorrectos por "tratar de reinventar la rueda" y con  seguir buenas praxis de web semántica.

Las preguntas son, esencialmente, las siguientes:

* Las entidades que pretendo representar en mi dominio, ¿están ya representadas en algún otro grafo de conocimiento? Si es así, ¿debo utilizar sus URIs (identificadores) en mi propio grafo?

* Las clases de mis entidades, ¿han sido definidas ya, aunque sea de manera aproximada, en otra ontología? Si es así, ¿debo utilizar esas clases en mis grafos?

* Las propiedades que unen conceptos en mi grafo, ¿ya han sido definidas en otras ontologías? Si lo han sido, son compatibles en dominio y rango con mi elección de clases? ¿Debería utilizarlas?

* Para todas aquellas entidades, clases o propiedades que deba crear porque no existen en otros grafos, o existen pero he decidido no utilizarlas: ¿cuál debería ser mi esquema de URIs?


### Explorando un dominio de ejemplo: Pokemon


Comencemos por lo esencial. Vamos a coger entidades representativas de mi dominio e investigar si existen en fuentes conocidas. Sin duda, el tipo de entidad principal de mi dominio es la idea de "Pokemon". Como primer paso, debería investigar si existe una ontología sobre pokemon definida y, de existir, cuan completa, importante o correcta parece ser.

Cosas que se pueden intentar en este paso:
* Probar a poner palabras clave en Linked Open Vocabularies (https://lov.linkeddata.es/dataset/lov/) --> En este caso, aparece una mención muy lateral a pokemon.

* Probar a buscar ontologías o grafos en la Linked Open Data Could (https://lod-cloud.net/) --> En este caso, aparece algo, pero parece que el dataset ya no está disponible.

* Probar a buscar entidades representativas en grafos de conocimiento relevantes (Wikidata, DBpedia, etc.). En este caso, parece que hemos dado con listas completas de pokemon tanto en DBpedia como en Wikidata. Aquí tenemos dos vías de acción: por un lado, podríamos adoptar las URIs de cada pokemon en estos grafos como "propias" al definir nuestro datos. NO crearíamos ninguna URI nueva de ningún Pokemon. Por otro lado, podríamos mantener nuestras propias URIs y enlazar con la de estos grafos a través de declaraciones owl:sameAs/schema:sameAs.

* Probar en buscadares web palabras clave + ontología / RDF / grafo conocimiento. Generalmente, dará mejores resultados hacer estas búsquedas en ingles. En este caso, nos aparece una Pokemon ontology y un Pokemon Knowledge Graph limpios y exhaustivos. Quedaría por evaluar hasta qué punto se adapta a nuestras necesidades.


En base a los elementos encontrados, debemos de ser capaces de decidir cuál es la mejor estrategia posible para representar nuestros datos.

A nivel de instancia, en caso de haber encontrado definiciones de identificadores existentes para tus entidades, usar esos mismos identificadores en tu grafo no es estrictametne necesario desde el punto de vista técnico, pero sí puede ser conveniente. Casi siempre, hay pros y contras de adoptar una u otra decisión en este aspecto.

#### Caso de Wikidata

Parece estar La lista completa de Pokemon, pero las URIs de Wikidata son "opacas". En lugar de tener URIs con nombres reconocibles, pasaríamos a tener identificadorer terminados en Qxxxxx. Eso podría no ser conveniente para la usabilidad/legibilidad de tus datos.

La lista de Pokemon podría variar en un futuro. ¿Podemos estar seguros de que si se incorpora algún Pokemon nuevo este se añadirá a Wikidata? Es muy probable que sí, puesto que incluso nosotros mismos podemos provocar ese cambio editando la fuente. Eso sí, aunque sea poco probable, estaríamos expuestos a situaciones de bandalismo de datos en Wikidata, puesto que cualquier podría editar los items que nosotros mismos editamos. Con todo, sería nrazonable pensar que podemos asumir que esa lista puede actualizarse de acuerdo a nuestras necesidades.

No obstante, si en lugar de usar la URIs de Wikidata desarrollasemos nuestras propias URIs y las enlazasemos con Qs de Wikidata mediante tirpletas owl:sameAs/schema:sameAs, podríamos mantener URIs legibles, mantenernos enlazados a una fuente tan relevante como Wikidata y podríamos seguir creando URIs de nuevos pokemon sin depender de que estos estén o no en Wikidata. En el momento en que entren, añadiríamos una tripleta owl:sameAs para enlazar nuestra URI y la de Wikidata. El unico contra de esta opción es que tendríamos que añadir esas tripletas owl:sameAs a nuestros datos --> dataset más grande para contener la misma información y las consultas federadas se volverán algo más complejas.

La diferencia entre owl:sameAs y schema:sameAs es sutil, pero existe. owl:sameAs implica (o debería implicar) una equivalencia lógica absoluta. Es decir que las URIs unidas por un owl:sameAs debería poder ser totalmente intercambiables en sus contexto sin que esto signifique que se producirá ninguna contradicción lógica en cuestiones como dominio, rango o cardinalidad que no sucediese antes. schema:sameAs es "más relajado". Indica que las URIs están referidas a la misma entidad o concepto real, pero no necesariamente que estas URIs sean equivalentes y compatibles desde el punto de vista lógico a todos los niveles.

#### Caso de DBpedia

A efectos prácticos de tomar esta decisión respecto a las URIs, Wikidata y DBpedia tienen en común ser proyectos de reconocido prestigio y muy centrales. Merece la pena enlazarse con ellos. Tienen estos dos rasgos diferentes de especial relevancia para la cuestión: las URIs de DBpedia no son opacas y tampoco las actualiza la comunidad (o no directamente al menos).

Esto quiere decir que si adoptamos las URIs de DBpedia para deescribir pokemon en nuestro grafo, ganaríamos en legibilidad. Pero, eso sí, perderíamos en capacidad de reacción. Si salen nuevos Pokemon y quisiésemos meter datos sobre ellos, no tenemos control en absoluto respecto a cuando (o si quiera si ocurrirá) serán introducidas URIs para esos Pokemon en DBpedia. Para circunvalar ese problema, podríamos hacer mezcla de URIs de DBpedia con URIs propias allá donde haya una nueva criatura no incluida en la fuente. Pero esto queda "feo".

Las ventajas e inconvenientes de incluir sentencias owl:SameAs enlazando URIs propias de nuestro espacio de nombre y URIs de DBpedias son las mismas que en el caso de Wikidata.

#### Caso de Pokemon ontology

Tanto la ontología como el grafo de conocimiento parecen bien diseñados. Aunque no se encuentran indicios que indiquen que es un proyecto relevante en el ecosistema LOD.

Si la ontología está, efectivamente, correcta y completa para nuestros fines, hay que poner en una balanza el trabajo que nos ahorra usarla a nivel de modelado de clases y propiedades o incluso de aprovechamiento de información disponible versus el riesgo de que deje de mantenerse.

Ejemplo: supón que das por bueno todo lo que ahí está escrito, lo que te ahorra hacer un montón de trabajo de modelado y de recopilación de información. Si tu trabajo original requería hacer pokemon + regiones + movimientos + gimnasios, quizá ahora sea sólo gimnasios, porque lo demás ya está ahí. Tendría todo el sentido en ese caso adoptar las URIs de ese grafo y simplemente añadir información a los datos existentes. No se trataría de copiar y pegar esos datos existentes, sino en confiar en su veracidad y enlazarlos desde tus propios datos. Ahora bien: ¿y si hubiese que corregir los datos? No tendríamos en absoluto control sobre las ediciones ni ninguna garantía de que estas vayan a llevarse a cabo.

En un escenario real, donde pretendemos publicar datos importantes para X objetivo, una vía deacción válida sería contactar personalmente con la persona o entidad que mantiene la ontología. Conocer sus intenciones y posibilidades respecto a la estabilidad de los datos, su mantenimiento en caso necesario o incluso la posibilidad de pasar a formar parte del grupo de personas con capacidad para mantener la aontología.


### Esquema de URIs

Decidas lo que decidas sobre qué datos reusar, muy probablemente vas a generar nuevas URIs. Tengas o no intención de dejar públicos tus datos, deberías siempre considerar tener un esquema de URIs consistente, bien organizado, centrado en el dominio y tan natural y sencillo como sea posible.

En primer lugar, el inicio de la URI, a no ser que estemos usando example.org, debería hacer referencia a algún dominio sobre el que tuvieses control, o sea pertinente en el caso que corresponda.

Por ejemplo, yo, si tengo que publicar datos consecuencia de mi actividad investigadora, lo haría en el dominio http://weso.es//, correpondiente a mi grupo de investigación, sobre el que el grupo tiene pleno control. Si estuviese trabajando para otra empresa, el dominio de su empresa.
En vuestro caso, en el contexto de esta asignatura, podría tener sentido, por ejemplo, publicar datos en el dominio de uniovi, a pesar de que no está bajo vuestro control, pero con más niveles de profundidad. Por ejemplo, tendría sentido hacer algo como esto:







In [ ]:
ttl_8 = """

PREFIX miw : <http://uniovi.es/miw/ws/tu_tema#>

miw:charmander a miw:Pokemon .
"""
# miw por master de ingeniería web, ws por web semántica, y alguna palabra clave
# de tu tema. Esto ayuda a asegurar que ningún compañero publica algo en miw/ws/
# que tú también hayas publicado y se repitan las URIs. Una alternativa más segura
# al tema sería tu UO. Aunque es menos "elegante"

### Conclusión trasversal

Llega un punto en el que tenemos que abordar estos temas con pragmatismo. Quitarnos el gorro de ontólogos y ponernos el de ingenieros. La meta, en general, es clara: producir datos que "sirvan". Para X aplicación, para Y análisis. No todo tiene que ser LOD cloud impecable o ser compatible con razonadores OWL. Hemos de buscar calidad de los datos, pero dentro de nuestra capacidad, recursos y objetivos.

¿Pretendemos simplemente crear una web adecuada para SEO? El alcance de todas estas preguntas se reduce a cómo meter schema.org donde proceda.

¿Pretendemos hacer un grafo RDF pero no dejarlo expuesto públicamente, sino que me sirva de backend para cierta aplicación? No hay que ser "chambones", pero tampoco darle más vueltas de las necesarias a detalles superfluos. Discusiones sobre owl:sameAs vs schema:sameAs, investigaciones sobre cuántas fuentes LOD es conveniente enlazar, consecuencias lógicas de usar una u otro ontología, pasan a un segundísimo plano. Más importa que la información pertinente para tu dominio esté ahí y sea fácil de mantener y consultar.

¿Pretendemos modelizar un dominio de conocimiento y exponer nuestro grafo/ontología en un endpoint o a través de dumps? Sí deberíamos dar una vuelta a todo lo planteado en este Jupyter.

De nuevo, el punto donde hay que parar de discutir depende en mucha medida de tus objetivos y recursos.